In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import sys
import os
#Perform the necessary pull of some helper functions.

from lmfit import Model
import scipy
import matplotlib.pyplot as plt
# To match plot styles in the draft.
plt.rcParams['xtick.direction']='in'
plt.rcParams['ytick.direction']='in'
plt.rcParams['xtick.minor.visible']=True
plt.rcParams['ytick.minor.visible']=True
plt.rcParams['xtick.top']=True
plt.rcParams['ytick.right']=True
plt.rcParams['font.size']=10
plt.rcParams['xtick.labelsize']=10
plt.rcParams['ytick.labelsize']=10
plt.rcParams['text.usetex']=False
plt.rcParams['font.family']='serif'

import bnrof
import MDUtils as mdu
from bnrof import IEXY_data
from bnrof import Material
from mantid.geometry import CrystalStructure, ReflectionGenerator, ReflectionConditionFilter

FrameworkManager-[Notice] Welcome to Mantid 6.7.0
FrameworkManager-[Notice] Please cite: http://dx.doi.org/10.1016/j.nima.2014.07.029 and this release: http://dx.doi.org/10.5286/Software/Mantid6.7
DownloadInstrument-[Notice] All instrument definitions up to date


## The first step in the analysis of the SEQUOIA data is the normalization to the available structural Bragg peaks. 

This is necessary to extract quantities like the total frozen / dynamic moment and exchange interaction estimates later. The theory behind this is described in the appendix of the main text. 

In [2]:
Alum = Material('Aluminum.cif',suppress_print=True,nist_data='nist_scattering_table.txt')
BNRO_mantid = CrystalStructure("5.75733 5.75733 28.58700","R -3 m", "Ba 0 0 0.12933 1.0 0.015;\
                                                                    Ba 0 0 0.28678 1.0 0.011;\
                                                                    Nb 0 0 0 1.0 0.004;\
                                                                    Ru 0 0 0.5 1.0 0.016;\
                                                                    Ru 0 0 0.4122 1.0 0.011;\
                                                                    O 0.4923 0.50770 0.12244 1.0 0.022;\
                                                                    O 0.50552 0.4945 0.29332 1.0 0.013")
BNRO_th = Material('Ba4NbRu3O12_refined-structure.cif',suppress_print=True)

Qslice50 = bnrof.genQslice(0,4,100)
Eslice50 = bnrof.genEslice(-50,50,100)


Qslice25 = bnrof.genQslice(0,4,100)
Eslice25 = bnrof.genEslice(-25,25,100)

Qslice10 = bnrof.genQslice(0,2.5,60)
Eslice10 = bnrof.genEslice(-10,10,40)

#Start importing data files. MT means empty can. 
lowT_MT_f10 = 'SEQ Data Files/MTCenterPos_10p50_4p04.nxspe'
highT_MT_f10 = 'SEQ Data Files/MTCenterPos_10p50_150p01.nxspe'
lowT_samp_f10 ='SEQ Data Files/CenterPos_10p50_3p98.nxspe'
highT_samp_f10 = 'SEQ Data Files/CenterPos_10p50_150p00.nxspe'

# nxspes without multiphonon
lowT_MT_f25 = 'SEQ Data Files/MTCenterPos_25p00_4p00.nxspe'
highT_MT_f25 = 'SEQ Data Files/MTCenterPos_25p00_300p00.nxspe'
lowT_samp_f25 ='SEQ Data Files/CenterPos_25p00_3p95.nxspe'
highT_samp_f25 = 'SEQ Data Files/CenterPos_25p00_300p11.nxspe'

lowT_MT_f50 = 'SEQ Data Files/MTCenterPos_50p00_4p30.nxspe'
highT_MT_f50 = 'SEQ Data Files/MTCenterPos_50p00_300p00.nxspe'
lowT_samp_f50 ='SEQ Data Files/CenterPos_50p00_4p22.nxspe'
highT_samp_f50 = 'SEQ Data Files/CenterPos_50p00_300p00.nxspe'

BNRO_mantid = CrystalStructure("5.75733 5.75733 28.58700","R -3 m", "Ba 0 0 0.12933 1.0 0.015;\
                                                                    Ba 0 0 0.28678 1.0 0.011;\
                                                                    Nb 0 0 0 1.0 0.004;\
                                                                    Ru 0 0 0.5 1.0 0.016;\
                                                                    Ru 0 0 0.4122 1.0 0.011;\
                                                                    O 0.4923 0.50770 0.12244 1.0 0.022;\
                                                                    O 0.50552 0.4945 0.29332 1.0 0.013")

BNRO_th.formula_units=3.0
BNRO=BNRO_th

MD_25_4 = bnrof.import_files_to_MD(lowT_samp_f25,MT_arr=lowT_MT_f25,Q_slice=Qslice25,\
                                        E_slice=Eslice25,self_shield=1.0,numEvNorm=True).clone()
MD_25_300 = bnrof.import_files_to_MD(highT_samp_f25,MT_arr=highT_MT_f25,Q_slice=Qslice25,\
                                          E_slice=Eslice25,self_shield=1.0,numEvNorm=True).clone()
MD_50_4 = bnrof.import_files_to_MD(lowT_samp_f50,MT_arr=lowT_MT_f50,Q_slice=Qslice50,\
                                        E_slice=Eslice50,self_shield=1.0,numEvNorm=True).clone()
MD_50_300 = bnrof.import_files_to_MD(highT_samp_f50,MT_arr=highT_MT_f50,Q_slice=Qslice50,\
                                          E_slice=Eslice50,self_shield=1.0,numEvNorm=True).clone()
MD_10_4 = bnrof.import_files_to_MD(lowT_samp_f10,MT_arr=lowT_MT_f10,Q_slice=Qslice10,\
                                     E_slice=Eslice10,self_shield=1.0,numEvNorm=True).clone()
MD_10_150 = bnrof.import_files_to_MD(highT_samp_f10,MT_arr=highT_MT_f10,Q_slice=Qslice10,\
                                      E_slice=Eslice10,self_shield=1.0,numEvNorm=True).clone()


d_eff = 0.5 #in cm
#Simple absorption model below using tabulated values. Assumes d_eff
# is the average neutron path and that the neutron energy is the geometric
# mean of the incoming and outgoing energies. In BRNO the attenuation is always
# very small, less than 5%. 
MD_abs_25_4 = bnrof.correct_absorb_MD_Material(MD_25_4,25.0,BNRO_th,d_eff,False).clone()
MD_abs_25_300 = bnrof.correct_absorb_MD_Material(MD_25_300,25.0,BNRO_th,d_eff,False).clone()

MD_abs_50_4 = bnrof.correct_absorb_MD_Material(MD_50_4,50.0,BNRO_th,d_eff,False).clone()
MD_abs_50_300 = bnrof.correct_absorb_MD_Material(MD_50_300,50.0,BNRO_th,d_eff,False).clone()

MD_abs_10_4 = bnrof.correct_absorb_MD_Material(MD_10_4,10.0,BNRO_th,d_eff,False).clone()
MD_abs_10_150 = bnrof.correct_absorb_MD_Material(MD_10_150,10.0,BNRO_th,d_eff,False).clone()

scale_list = []
measurement_list = [MD_abs_25_4,MD_abs_25_300,MD_abs_50_4,MD_abs_50_300,MD_abs_10_4,MD_abs_10_150]
#Plot each
Ei_list = [25.0,25.0,50.0,50.0,10.5,10.5]
fwhm_list=[1.4,1.4,3.0,3.0,0.6,0.6]
temp_list = [4.0,300.0,4.0,300.0,4.0,150.0]

#scale dict from manual normalizations:
correctscales = np.array([2.56e1,2.67e1,2.132e1,2.201e1,3.738e2,3.759e2])*10.0

for i in range(len(measurement_list)):
    md=measurement_list[i]
    Ei=Ei_list[i]
    energy_fwhm = fwhm_list[i]
    if Ei<11.5:
        bannedQ = [[0,0.5],[2.0,3.0]]
    elif Ei==25.0:
        bannedQ = [[0,0.5],[3.2,5.0]]
    else:
        bannedQ=[[0,0.5]]
    if i in [0]:
        pltresults=True
    else:
        pltresults=True
    datscale = bnrof.get_md_Material_bragg_normalization_factors(md,Ei,\
                                            Ei*0.05,BNRO_mantid,BNRO_th,aluminum_material=Alum,\
                                            scale_guess=5.0e2,peak_width_guess=0.025,alum_scale_guess=0.0,sample_mass=1.0,\
                                            allow_aluminum=False,banned_Q_regimes=[[0.0,1.0],[3.2,6.0]],fit_method='powell',\
                                            allow_q_shift=False,plot_results=pltresults,
                                            plot_savedir=f"SEQ_figs/SEQ_norm_Ei_{Ei:.2f}_T_{temp_list[i]:.2f}.pdf")
    scale_list.append(datscale['result_scale'])
    print(f"For Ei={Ei}, extracted scale={datscale['result_scale']:.2e}")
    print(f"Estimated Correct scale = {correctscales[i]:.2e}")

 Material.formula_weight=(val)
LoadNXSPE-[Notice] LoadNXSPE started
LoadNXSPE-[Notice] LoadNXSPE successful, Duration 0.08 seconds
LoadNXSPE-[Notice] LoadNXSPE started
LoadNXSPE-[Notice] LoadNXSPE successful, Duration 0.06 seconds
CreateSingleValuedWorkspace-[Notice] CreateSingleValuedWorkspace started
CreateSingleValuedWorkspace-[Notice] CreateSingleValuedWorkspace successful, Duration 0.00 seconds
Divide-[Notice] Divide started
Divide-[Notice] Divide successful, Duration 0.01 seconds
CreateSingleValuedWorkspace-[Notice] CreateSingleValuedWorkspace started
CreateSingleValuedWorkspace-[Notice] CreateSingleValuedWorkspace successful, Duration 0.00 seconds
Divide-[Notice] Divide started
Divide-[Notice] Divide successful, Duration 0.00 seconds
ConvertToMD-[Notice] ConvertToMD started
ConvertToMD-[Notice] ConvertToMD successful, Duration 0.03 seconds
ConvertToMD-[Notice] ConvertToMD started
ConvertToMD-[Notice] ConvertToMD successful, Duration 0.03 seconds
BinMD-[Notice] BinMD started
BinM